> 注意这里作者只是简单的介绍了几种函数的用法，但并未告诉我们为什么要这样做。


OutParser 的价值我们可以用一个例子快速说明，我们简单调用一个 LLM 请求


In [1]:
import { ChatOpenAI } from "@langchain/openai"
import { HumanMessage } from "@langchain/core/messages"

const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

await model.invoke([
  new HumanMessage("Tell me a joke"),
])


AIMessage {
  "id": "chatcmpl-B7viOcadu7RlCe19H3gUV5Z7oNpxV",
  "content": "Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 11,
      "completionTokens": 17,
      "totalTokens": 28
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "completion_tokens": 17,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens": 11,
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      },
      "total_tokens": 28
    },
    "system_fingerprint": "fp_b705f0c291"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 17,
    "input_tokens": 11,
    "total_tokens": 28,
    "input_token_details": {
     

这显然是人类不可读的，我们没办法把这样的对象去发送给用户。虽然这个结果是代码可以处理的，但我们每次都要写这样的 `dirty` 代码去提取其中我们需要的 `content` 或者未来需要的其他属性会很繁琐。而且未来可能会使用多种模型来替代 OpenAI 的模型，我们每次都适配不同的模型的 API 输出去提取需要的内容，也会很麻烦。

所以这就是 OutputParser 的意义之一，langchain 封装了一系列的解析大模型 API 返回结果的工具让我们方便的使用。当然，并不限于解析大模型的输出结果，也能通过 Parser 去指定 LLM 返回的格式，让我们逐步来学习。

## String Output Parser

我们接着上面的例子，如果我们只需要大模型的文本输出，就可以通过 `StringOutputParser` 获取其中的文本内容


In [2]:
import { StringOutputParser } from "@langchain/core/output_parsers"

const parser = new StringOutputParser()

const chain = model.pipe(parser)

await chain.invoke([
  new HumanMessage("Tell me a joke"),
])


"Why don't scientists trust atoms?\n\nBecause they make up everything!"

这是最简单的 Parser，提出 API 返回的文本数据（也就是 content ）部分。对比我们直接自己解析，langchain 内部会有错误处理和 stream 等支持。

通过这个简单 api，方便大家理解 output parser 的其中一个意义 -- 解析大模型的输出

## StructuredOutputParser

Output Parser 的另一个意义就是引导模型以你需要的格式进行输出，部分 Parser 会内置一些预先设计好的 prompt 对模型进行引导。 听起来不太好理解，我们少废话，直接看 code 更容易理解。

我们构建一个回答问题，并且会提供对应的证据和可信度评分


In [3]:
import { StructuredOutputParser } from "langchain/output_parsers"
import { PromptTemplate } from "@langchain/core/prompts"

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
})

console.log(parser.getFormatInstructions())


You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

定义这个 praser 的时候，我们需要指定我们需要的 Json 输出的 key 和对应的描述。注意这里的描述要写完整，包括你的要求的格式（比如我们这里写的格式是百分数），越清晰 LLM 越能返回给你需要的数值。


在使用这个 parser 之前，我们先看看 magic 是如何发生的，langchain 植入了什么样的 prompt，我们打印出来看看，也学习一下其中 prompt 的写法，让我们来逐段分析这个指令

- 先告诉 LLM 输出的类型为 "JSON Schema"
- 其次，使用 few-shot，也就是用示例告诉 LLM 什么是 JSON Schema，什么情况会被解析成功，什么情况不会被解析成功
- 然后，再次强调类型的重要性，输出必须遵循给定的 JSON Schema 实例，确保所有字段严格匹配 Schema 中的定义，没有额外的属性，也没有遗漏的必需属性。并强调需要注意细节，比如不要在 JSON 对象中添加多余的逗号，这可能会导致解析失败。这些 prompt 质量非常高，把在该任务中大模型容易出现的问题都进行了强调，可以有效的保证输出的质量。
- 最后才是给出，我们指定的 JSON 格式和对应的描述

通过这样一系列的 prompt，就能保证大模型以指定的格式输出，我们完成 Chain 的其他部分看看效果


In [4]:
const prompt = PromptTemplate.fromTemplate(
  "尽可能的回答用的问题 \n{instructions} \n{question}",
)

const chain = prompt.pipe(model).pipe(parser)

const res = await chain.invoke({
  question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
  instructions: parser.getFormatInstructions(),
})

console.log(res)


{
  answer: "蒙娜丽莎的作者是列奥纳多·达·芬奇，绘制于1503年至1506年之间。",
  evidence: "根据艺术历史学家的研究，蒙娜丽莎是在意大利文艺复兴时期由达·芬奇创作的最著名画作之一。",
  confidence: "95%"
}


这样，我们就完成了强制大模型按照我们需求和格式进行输出的 output parser。经过这个 demo 相信你也会理解，parser 不止是对大模型的输出进行处理，也有引导大模型按照给定格式输出的能力，并且内置了一些错误处理的能力，更容易在生产环境进行部署。


In [5]:
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers"

const parser = new CommaSeparatedListOutputParser()

console.log(parser.getFormatInstructions())


Your response should be a list of comma separated values, eg: `foo, bar, baz`


因为输出的格式更简单，所以这里引导性的 prompt 也更容易理解


In [6]:
const prompt = PromptTemplate.fromTemplate(
  "列出3个 {country} 的着名的互联网公司.\n{instructions}",
)

const chain = prompt.pipe(model).pipe(parser)

const response = await chain.invoke({
  country: "America",
  instructions: parser.getFormatInstructions(),
})

response


[ "Google", "Facebook", "Amazon" ]

这里可以看到返回了一个数组


## Auto Fix Parser

让我们更进一步，把 LLM 进一步引入到 output parser 中，对于部分对输出质量要求更高的场景，如果出现了输出不符合要求的情况，我们希望的不是让 LLM 反复输出（可能每次都是错的），因为 LLM 并没有意识到自己的错误。所以我们需要把报错的信息返回给 LLM，让他理解错在哪里，应该怎么修改。

首先，我们需要使用 zod，一个用来验证 js/ts 中类型是否正确的库。先使用 zod 定义一个我们需要的类型，这里我们指定了评分需要是一个数字，并且是 [0, 100] 的数字


In [1]:
import { z } from "zod"
import {
  OutputFixingParser,
  StructuredOutputParser,
} from "langchain/output_parsers"
import { PromptTemplate } from "@langchain/core/prompts"
import { ChatOpenAI } from "@langchain/openai"
import { HumanMessage } from "@langchain/core/messages"

const schema = z.object({
  answer: z.string().describe("用户问题的答案"),
  confidence: z.number().min(0).max(100).describe(
    "问题答案的可信度评分，满分 100",
  ),
})


然后我们先使用正常的方式，使用 zod 来创建一个 `StructuredOutputParser`


In [ ]:
const parser = StructuredOutputParser.fromZodSchema(schema)
const prompt = PromptTemplate.fromTemplate(
  "尽可能的回答用的问题 \n{instructions} \n{question}",
)
const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

const chain = prompt.pipe(model).pipe(parser)
const res = await chain.invoke({
  question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
  instructions: parser.getFormatInstructions(),
})

console.log(res)


{
  answer: "蒙娜丽莎的作者是列奥纳多·达·芬奇。这幅画大约是在1503年至1506年之间绘制的。",
  confidence: 95
}


然后，我们尝试构造一个可以根据 zod 定义以及错误的输出，来自动修复的 parser


In [4]:
const wrongOutput = {
  "answer":
    "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  "sources": "90%",
}

const fixParser = OutputFixingParser.fromLLM(model, parser)
const output = await fixParser.parse(JSON.stringify(wrongOutput))


In [5]:
output


{ answer: "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。", confidence: 90 }

然后，我们定义一个，数值超出限制的错误


In [6]:
const wrongOutput = {
  "answer":
    "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  "sources": "-1",
}

const fixParser = OutputFixingParser.fromLLM(model, parser)
const output = await fixParser.parse(JSON.stringify(wrongOutput))

output


{ answer: "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。", confidence: 95 }

~~可以看到，`OutputFixingParser` 在两次都修复成了 90，这显然是不符合事实的，让我们看一下其内置的 prompt~~
似乎是由于模型进步的原因，输出了正确的结果。


In [7]:
console.log(fixParser.getFormatInstructions())


You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

可以看到，这是一个纯粹的 JSON 格式处理，并不会在意其中的语意和用户的问题，所以需要在合适的时机用合适的方式去修复。 当然这个工具不止是对 LLM 格式化输出的修复，也可以修复任何场景下的 JSON 问题。

当然，可能会有朋友问，如果我把用户的问题也给 `fixParser`，这样不就得到一个正确的答案和正确的格式了么？ 在我们的 demo 中当然是可以的，但实际工程中，引导 llm 返回数据的 prompt 可能非常巨大，非常消耗 token，我们使用 `fixParser` 就是用较少的成本去修复这个输出，来节约重复调用的成本。所以把原文也给 `fixParser` 的话，就达不到成本节约的目的了。

在进一步节约成本的背景下，我们是可以用对 GPT4 的错误输出用 GPT3.5 的 fixer 来修复，甚至是用一些开源模型来进行修复，因为在这个场景下，并不需要模型具有太高的质量，通过多模型的协同来降低成本。
